In [10]:
from mxnet import autograd, nd
from mxnet.gluon import data as gdata
from mxnet.gluon import nn
from mxnet import init
from mxnet.gluon import loss as gloss
from mxnet import gluon

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

In [4]:
batch_size = 10
dataset = gdata.ArrayDataset(features, labels)
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)

In [5]:
for X, y in data_iter:
    print(X, y)
    break


[[ 0.27938566 -0.09736729]
 [-1.8847244   2.6387691 ]
 [ 0.78679943 -0.31629395]
 [-0.21165672 -1.9151295 ]
 [ 1.0041832   0.19835497]
 [ 1.1747298   1.1240152 ]
 [-1.087749   -0.6615745 ]
 [-1.1049094   0.44587043]
 [-0.3480856   0.37118176]
 [ 1.934676   -0.28565592]]
<NDArray 10x2 @cpu(0)> 
[ 5.09013   -8.530067   6.840453  10.279892   5.5313396  2.720426
  4.265578   0.4968183  2.236942   9.036529 ]
<NDArray 10 @cpu(0)>


In [11]:
net = nn.Sequential()
net.add(nn.Dense(1))
net.initialize(init.Normal(sigma=0.01))
loss = gloss.L2Loss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

In [12]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss 0.035193
epoch 2, loss 0.000128
epoch 3, loss 0.000049


In [13]:
dense = net[0]
true_w, dense.weight.data()

([2, -3.4], 
 [[ 1.9997255 -3.3992538]]
 <NDArray 1x2 @cpu(0)>)

In [15]:
true_b, dense.bias.data()

(4.2, 
 [4.199734]
 <NDArray 1 @cpu(0)>)

In [16]:
help(dense.weight)

Help on Parameter in module mxnet.gluon.parameter object:

class Parameter(builtins.object)
 |  Parameter(name, grad_req='write', shape=None, dtype=<class 'numpy.float32'>, lr_mult=1.0, wd_mult=1.0, init=None, allow_deferred_init=False, differentiable=True, stype='default', grad_stype='default')
 |  
 |  A Container holding parameters (weights) of Blocks.
 |  
 |  :py:class:`Parameter` holds a copy of the parameter on each :py:class:`Context` after
 |  it is initialized with ``Parameter.initialize(...)``. If :py:attr:`grad_req` is
 |  not ``'null'``, it will also hold a gradient array on each :py:class:`Context`::
 |  
 |      ctx = mx.gpu(0)
 |      x = mx.nd.zeros((16, 100), ctx=ctx)
 |      w = mx.gluon.Parameter('fc_weight', shape=(64, 100), init=mx.init.Xavier())
 |      b = mx.gluon.Parameter('fc_bias', shape=(64,), init=mx.init.Zero())
 |      w.initialize(ctx=ctx)
 |      b.initialize(ctx=ctx)
 |      out = mx.nd.FullyConnected(x, w.data(ctx), b.data(ctx), num_hidden=64)
 |  
 